In [29]:
import pandas as pd

# Data for the selected food items
food_data = pd.DataFrame({
    "FOOD ITEM": ["Crumpet, from white flour, toasted", 
                  "Doughnut, iced", 
                  "Pancake, plain, homemade", 
                  "Bean, broad, fresh, boiled, drained"],
    "PROTEIN": [5.60, 6.70, 7.40, 7.40],
    "CARBS": [41.60, 48.20, 30.30, 2.40],
    "FATS": [0.80, 15.90, 7.60, 0.50],
    "CALORIES": [201.00, 366.10, 220.80, 58.90],
    "QUANTITY": [100, 100, 100, 100],  # initial quantity in grams
    "UNIT": ["G", "G", "G", "G"]
})

# User's nutritional targets for breakfast (30% of daily intake)
target_calories = 593.55
target_protein = 16.8
target_carbs = 94.490625
target_fats = 16.4875

# Calculate total nutrients for the selected food items (default quantities)
total_protein = food_data["PROTEIN"].sum()
total_carbs = food_data["CARBS"].sum()
total_fats = food_data["FATS"].sum()
total_calories = food_data["CALORIES"].sum()

# Calculate scaling factors for each nutrient to meet the breakfast targets
protein_scale = target_protein / total_protein if total_protein > 0 else 0
carbs_scale = target_carbs / total_carbs if total_carbs > 0 else 0
fats_scale = target_fats / total_fats if total_fats > 0 else 0
calories_scale = target_calories / total_calories if total_calories > 0 else 0

# Apply the smallest scale factor to all items to meet the most limiting nutrient target
scaling_factor = min(protein_scale, carbs_scale, fats_scale, calories_scale)

# Adjust quantities of each food item
food_data["ADJUSTED QUANTITY"] = food_data["QUANTITY"] * scaling_factor

# Recalculate total macros with adjusted quantities
food_data["ADJUSTED PROTEIN"] = food_data["PROTEIN"] * scaling_factor
food_data["ADJUSTED CARBS"] = food_data["CARBS"] * scaling_factor
food_data["ADJUSTED FATS"] = food_data["FATS"] * scaling_factor
food_data["ADJUSTED CALORIES"] = food_data["CALORIES"] * scaling_factor

# Output the adjusted meal plan
print("Adjusted Meal Plan for Breakfast:")
print(food_data[["FOOD ITEM", "ADJUSTED QUANTITY", "ADJUSTED PROTEIN", "ADJUSTED CARBS", "ADJUSTED FATS", "ADJUSTED CALORIES"]])

# Sum the adjusted macros
total_adjusted_protein = food_data["ADJUSTED PROTEIN"].sum()
total_adjusted_carbs = food_data["ADJUSTED CARBS"].sum()
total_adjusted_fats = food_data["ADJUSTED FATS"].sum()
total_adjusted_calories = food_data["ADJUSTED CALORIES"].sum()

print("\nTotal Adjusted Macros for Breakfast:")
print(f"Calories: {total_adjusted_calories:.2f}")
print(f"Protein: {total_adjusted_protein:.2f}g")
print(f"Carbs: {total_adjusted_carbs:.2f}g")
print(f"Fats: {total_adjusted_fats:.2f}g")


target_calories = 593.55
target_protein = 16.8
target_carbs = 94.490625
target_fats = 16.4875


Adjusted Meal Plan for Breakfast:
                             FOOD ITEM  ADJUSTED QUANTITY  ADJUSTED PROTEIN  \
0   Crumpet, from white flour, toasted           61.99262          3.471587   
1                       Doughnut, iced           61.99262          4.153506   
2             Pancake, plain, homemade           61.99262          4.587454   
3  Bean, broad, fresh, boiled, drained           61.99262          4.587454   

   ADJUSTED CARBS  ADJUSTED FATS  ADJUSTED CALORIES  
0       25.788930       0.495941         124.605166  
1       29.880443       9.856827         226.954982  
2       18.783764       4.711439         136.879705  
3        1.487823       0.309963          36.513653  

Total Adjusted Macros for Breakfast:
Calories: 524.95
Protein: 16.80g
Carbs: 75.94g
Fats: 15.37g


In [33]:
import random
import math

# Define the food items and their nutritional information per 100g
food_items = [
    {
        "name": "Crumpet, from white flour, toasted",
        "protein": 5.60,     # g
        "carbs": 41.60,      # g
        "fats": 0.80,        # g
        "calories": 201.00   # kcal
    },
    {
        "name": "Doughnut, iced",
        "protein": 6.70,      # g
        "carbs": 48.20,       # g
        "fats": 15.90,        # g
        "calories": 366.10    # kcal
    }
]

# User's nutritional targets for breakfast
target_nutrients = {
    "calories": 593.55,   # kcal
    "protein": 16.8,      # g
    "carbs": 94.49,       # g
    "fats": 16.49         # g
}

# GA Parameters
POPULATION_SIZE = 100
GENERATIONS = 100
TOURNAMENT_SIZE = 5
CROSSOVER_RATE = 0.8
MUTATION_RATE = 0.2
# Define realistic portion size ranges (in grams)
MIN_PORTION = 0    # grams
MAX_PORTION = 300  # grams

# Initialize population with random quantities
def initialize_population():
    population = []
    for _ in range(POPULATION_SIZE):
        chromosome = []
        for _ in food_items:
            qty = random.uniform(50, 200)  # Initialize with quantities between 50g and 200g
            chromosome.append(qty)
        population.append(chromosome)
    return population

# Fitness function: sum of squared deviations
def fitness(chromosome):
    total_calories = 0
    total_protein = 0
    total_carbs = 0
    total_fats = 0
    for gene, food in zip(chromosome, food_items):
        factor = gene / 100.0
        total_calories += food["calories"] * factor
        total_protein += food["protein"] * factor
        total_carbs += food["carbs"] * factor
        total_fats += food["fats"] * factor
    # Calculate squared deviations
    calorie_dev = (total_calories - target_nutrients["calories"]) ** 2
    protein_dev = (total_protein - target_nutrients["protein"]) ** 2
    carbs_dev = (total_carbs - target_nutrients["carbs"]) ** 2
    fats_dev = (total_fats - target_nutrients["fats"]) ** 2
    # Sum of squared deviations
    total_deviation = calorie_dev + protein_dev + carbs_dev + fats_dev
    return math.sqrt(total_deviation)  # Lower is better

# Tournament selection
def tournament_selection(population, scores):
    selected = []
    for _ in range(POPULATION_SIZE):
        tournament = random.sample(list(zip(population, scores)), TOURNAMENT_SIZE)
        tournament.sort(key=lambda x: x[1])  # Sort by fitness (lower is better)
        selected.append(tournament[0][0])
    return selected

# Crossover (single-point)
def crossover(parent1, parent2):
    if random.random() < CROSSOVER_RATE:
        point = random.randint(1, len(parent1)-1)
        child1 = parent1[:point] + parent2[point:]
        child2 = parent2[:point] + parent1[point:]
        return child1, child2
    else:
        return parent1.copy(), parent2.copy()

# Mutation (random adjustment)
def mutate(chromosome):
    for i in range(len(chromosome)):
        if random.random() < MUTATION_RATE:
            # Apply a small mutation: adjust quantity by +/-10%
            mutation_factor = random.uniform(0.98, 1.02)
            new_qty = chromosome[i] * mutation_factor
            # Ensure quantities are within realistic bounds
            new_qty = max(MIN_PORTION, min(new_qty, MAX_PORTION))
            chromosome[i] = new_qty
    return chromosome

# GA Main Loop
def genetic_algorithm():
    population = initialize_population()
    for generation in range(GENERATIONS):
        # Evaluate fitness
        scores = [fitness(chrom) for chrom in population]
        # Check for convergence or optimal fitness (optional)
        # Selection
        selected = tournament_selection(population, scores)
        # Create next generation
        next_generation = []
        for i in range(0, POPULATION_SIZE, 2):
            parent1 = selected[i]
            if i+1 < POPULATION_SIZE:
                parent2 = selected[i+1]
            else:
                parent2 = selected[0]
            child1, child2 = crossover(parent1, parent2)
            child1 = mutate(child1)
            child2 = mutate(child2)
            next_generation.extend([child1, child2])
        # Replace population with next generation
        population = next_generation[:POPULATION_SIZE]
        # Optionally print progress
        if (generation+1) % 10 == 0 or generation == 0:
            best_score = min(scores)
            avg_score = sum(scores) / len(scores)
            print(f"Generation {generation+1}: Best Fitness = {best_score:.4f}, Avg Fitness = {avg_score:.4f}")
    # After all generations, find the best solution
    final_scores = [fitness(chrom) for chrom in population]
    best_index = final_scores.index(min(final_scores))
    best_chromosome = population[best_index]
    return best_chromosome, final_scores[best_index]

# Run the GA
best_solution, best_fitness_score = genetic_algorithm()

# Calculate the nutritional values of the best solution
def calculate_nutrients(chromosome):
    total_calories = 0
    total_protein = 0
    total_carbs = 0
    total_fats = 0
    for gene, food in zip(chromosome, food_items):
        factor = gene / 100.0
        total_calories += food["calories"] * factor
        total_protein += food["protein"] * factor
        total_carbs += food["carbs"] * factor
        total_fats += food["fats"] * factor
    return total_calories, total_protein, total_carbs, total_fats

best_calories, best_protein, best_carbs, best_fats = calculate_nutrients(best_solution)

# Print the results
print("\n=== Nutritional Targets for Breakfast ===")
print(f"Calories: {target_nutrients['calories']} kcal")
print(f"Protein: {target_nutrients['protein']} g")
print(f"Carbs: {target_nutrients['carbs']} g")
print(f"Fats: {target_nutrients['fats']} g")

print("\n=== Best Meal Plan Found ===")
for i, gene in enumerate(best_solution):
    print(f"{food_items[i]['name']}: {gene:.2f} g")
print("\n=== Total Nutritional Values of Best Meal Plan ===")
print(f"Calories: {best_calories:.2f} kcal")
print(f"Protein: {best_protein:.2f} g")
print(f"Carbs: {best_carbs:.2f} g")
print(f"Fats: {best_fats:.2f} g")
print(f"\nFitness Score: {best_fitness_score:.4f}")

# Optional: Compare with targets
print("\n=== Comparison with Targets ===")
print(f"Calories Deviation: {best_calories - target_nutrients['calories']:.2f} kcal")
print(f"Protein Deviation: {best_protein - target_nutrients['protein']:.2f} g")
print(f"Carbs Deviation: {best_carbs - target_nutrients['carbs']:.2f} g")
print(f"Fats Deviation: {best_fats - target_nutrients['fats']:.2f} g")


Generation 1: Best Fitness = 8.1914, Avg Fitness = 157.5996
Generation 10: Best Fitness = 4.3014, Avg Fitness = 5.3963
Generation 20: Best Fitness = 4.2634, Avg Fitness = 4.6683
Generation 30: Best Fitness = 4.2567, Avg Fitness = 4.6609
Generation 40: Best Fitness = 4.1621, Avg Fitness = 4.5855
Generation 50: Best Fitness = 4.2429, Avg Fitness = 4.6926
Generation 60: Best Fitness = 4.1431, Avg Fitness = 5.2257
Generation 70: Best Fitness = 4.1187, Avg Fitness = 4.6155
Generation 80: Best Fitness = 4.1061, Avg Fitness = 4.6959
Generation 90: Best Fitness = 4.0063, Avg Fitness = 4.8497
Generation 100: Best Fitness = 4.0058, Avg Fitness = 4.6285

=== Nutritional Targets for Breakfast ===
Calories: 593.55 kcal
Protein: 16.8 g
Carbs: 94.49 g
Fats: 16.49 g

=== Best Meal Plan Found ===
Crumpet, from white flour, toasted: 106.12 g
Doughnut, iced: 103.85 g

=== Total Nutritional Values of Best Meal Plan ===
Calories: 593.50 kcal
Protein: 12.90 g
Carbs: 94.20 g
Fats: 17.36 g

Fitness Score: 4.0

In [34]:
import pandas as pd

# Data for the selected food items
food_data = pd.DataFrame({
    "FOOD ITEM": ["Crumpet, from white flour, toasted", 
                  "Doughnut, iced", 
                  "Pancake, plain, homemade", 
                  "Bean, broad, fresh, boiled, drained"],
    "PROTEIN": [5.60, 6.70, 7.40, 7.40],
    "CARBS": [41.60, 48.20, 30.30, 2.40],
    "FATS": [0.80, 15.90, 7.60, 0.50],
    "CALORIES": [201.00, 366.10, 220.80, 58.90],
    "QUANTITY": [100, 100, 100, 100],  # initial quantity in grams
    "UNIT": ["G", "G", "G", "G"]
})

# User's nutritional targets for breakfast (30% of daily intake)
target_calories = 593.55
target_protein = 16.8
target_carbs = 94.490625
target_fats = 16.4875

# Calculate total nutrients for the selected food items (default quantities)
total_protein = food_data["PROTEIN"].sum()
total_carbs = food_data["CARBS"].sum()
total_fats = food_data["FATS"].sum()
total_calories = food_data["CALORIES"].sum()

# Calculate scaling factors for each nutrient to meet the breakfast targets
protein_scale = target_protein / total_protein if total_protein > 0 else 0
carbs_scale = target_carbs / total_carbs if total_carbs > 0 else 0
fats_scale = target_fats / total_fats if total_fats > 0 else 0
calories_scale = target_calories / total_calories if total_calories > 0 else 0

# Apply the smallest scale factor to all items to meet the most limiting nutrient target
scaling_factor = min(protein_scale, carbs_scale, fats_scale, calories_scale)

# Adjust quantities of each food item
food_data["ADJUSTED QUANTITY"] = food_data["QUANTITY"] * scaling_factor

# Recalculate total macros with adjusted quantities
food_data["ADJUSTED PROTEIN"] = food_data["PROTEIN"] * scaling_factor
food_data["ADJUSTED CARBS"] = food_data["CARBS"] * scaling_factor
food_data["ADJUSTED FATS"] = food_data["FATS"] * scaling_factor
food_data["ADJUSTED CALORIES"] = food_data["CALORIES"] * scaling_factor

# Output the adjusted meal plan
print("Adjusted Meal Plan for Breakfast:")
print(food_data[["FOOD ITEM", "ADJUSTED QUANTITY", "ADJUSTED PROTEIN", "ADJUSTED CARBS", "ADJUSTED FATS", "ADJUSTED CALORIES"]])

# Sum the adjusted macros
total_adjusted_protein = food_data["ADJUSTED PROTEIN"].sum()
total_adjusted_carbs = food_data["ADJUSTED CARBS"].sum()
total_adjusted_fats = food_data["ADJUSTED FATS"].sum()
total_adjusted_calories = food_data["ADJUSTED CALORIES"].sum()

print("\nTotal Adjusted Macros for Breakfast:")
print(f"Calories: {total_adjusted_calories:.2f}")
print(f"Protein: {total_adjusted_protein:.2f}g")
print(f"Carbs: {total_adjusted_carbs:.2f}g")
print(f"Fats: {total_adjusted_fats:.2f}g")


Adjusted Meal Plan for Breakfast:
                             FOOD ITEM  ADJUSTED QUANTITY  ADJUSTED PROTEIN  \
0   Crumpet, from white flour, toasted           61.99262          3.471587   
1                       Doughnut, iced           61.99262          4.153506   
2             Pancake, plain, homemade           61.99262          4.587454   
3  Bean, broad, fresh, boiled, drained           61.99262          4.587454   

   ADJUSTED CARBS  ADJUSTED FATS  ADJUSTED CALORIES  
0       25.788930       0.495941         124.605166  
1       29.880443       9.856827         226.954982  
2       18.783764       4.711439         136.879705  
3        1.487823       0.309963          36.513653  

Total Adjusted Macros for Breakfast:
Calories: 524.95
Protein: 16.80g
Carbs: 75.94g
Fats: 15.37g
